# Сервис для чтения книг

## Описание проекта

Компания приобрела крупный сервис для чтения книг по подписке. Задача — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Подключение к базе данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Исследуем таблицы, для этого выведем первые 5 строк каждой из них.

In [2]:
for table in ['books', 'authors', 'publishers', 'ratings', 'reviews']:
    query = f'''SELECT * FROM {table} LIMIT 5'''
    con=engine.connect()
    print(f'Таблица {table}')
    display(pd.io.sql.read_sql(sql=text(query), con = con))
    print('--------------------------------------------------------')

Таблица books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


--------------------------------------------------------
Таблица authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


--------------------------------------------------------
Таблица publishers


...

--------------------------------------------------------
Таблица ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


--------------------------------------------------------
Таблица reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


--------------------------------------------------------


## Запросы к базе

### Задание 1
Посчитайте, сколько книг вышло после 1 января 2000 года

In [3]:
query = '''
        SELECT COUNT(book_id)
        FROM books
        WHERE publication_date > '2000-01-01'
        '''
con=engine.connect()
display(pd.io.sql.read_sql(sql=text(query), con = con))

,count
0,819


### Задание 2
Для каждой книги посчитайте количество обзоров и среднюю оценку

In [4]:
query = '''
        SELECT title,
               COUNT(DISTINCT(review_id)) AS reviews_count,
               ROUND(AVG(rating), 2) AS avg_rating
        FROM books AS b
        LEFT JOIN reviews AS rev ON b.book_id = rev.book_id
        LEFT JOIN ratings AS rat ON b.book_id = rat.book_id
        GROUP BY b.book_id
        ORDER BY reviews_count DESC, avg_rating DESC
        '''
con=engine.connect()
display(pd.io.sql.read_sql(sql=text(query), con = con))

,title,reviews_count,avg_rating
0,Twilight (Twilight #1),7,3.66
1,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
2,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
3,The Book Thief,6,4.26
4,The Glass Castle,6,4.21
...,...,...,...
995,Disney's Beauty and the Beast (A Little Golden...,0,4.00
996,Leonardo's Notebooks,0,4.00
997,Essential Tales and Poems,0,4.00
998,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67


### Задание 3
Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [5]:
query = '''
        SELECT publisher,
               COUNT(b.book_id) AS books_count
        FROM publishers AS p
        LEFT JOIN books AS b ON p.publisher_id = b.publisher_id
        WHERE num_pages > 50
        GROUP BY publisher
        ORDER BY books_count DESC
        LIMIT 1
        '''
con=engine.connect()
display(pd.io.sql.read_sql(sql=text(query), con = con))

,publisher,books_count
0,Penguin Books,42


### Задание 4
Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [6]:
query = '''
        SELECT author,
               ROUND(AVG(rating), 2) AS avg_rating
        FROM authors AS a
        LEFT JOIN books AS b ON a.author_id = b.author_id
        LEFT JOIN ratings AS r ON b.book_id = r.book_id
        WHERE b.book_id IN (SELECT book_id
                            FROM ratings
                            GROUP BY book_id
                            HAVING COUNT(rating_id) >= 50)
        GROUP BY author
        ORDER BY avg_rating DESC
        LIMIT 1
        '''
con=engine.connect()
display(pd.io.sql.read_sql(sql=text(query), con = con))

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29


### Задание 5
Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [7]:
query = '''
            WITH reviews_count_table AS 
            (SELECT username,
                   COUNT(review_id) AS reviews_count
            FROM reviews 
            WHERE username IN (SELECT username
                               FROM ratings
                               GROUP BY username
                               HAVING COUNT(rating_id) > 48)
            GROUP BY username)
            
            SELECT AVG(reviews_count) AS avg_reviews_count
            FROM reviews_count_table
        '''
con=engine.connect()
display(pd.io.sql.read_sql(sql=text(query), con = con))

,avg_reviews_count
0,24.0


## Вывод

- После 1 января 2000 года вышло 819 книг
- Больше всего обзоров у Twilight - 7
- Издательство, которое выпустило наибольшее число книг толще 50 страниц, - Penguin Books
- Самая высокая средняя оценка книг у J.K. Rowling/Mary GrandPré
- Среднее количество обзоров на пользователя, который поставил более 48 оценок, - 24.